### Notes:
- nominal design performs well because momenta specifically based on current detector
    - different momenta will produce different optimal configurations
- need to pick momenta that are a priori interesting to optimize for

- also need better param bounds

- mupisepplow has lower cross validation score and is overall lower because it is harder to improve
    - mupisepphigh is just "whether particle makes it through all layers or not"
    - mupisepplow is more difficult because muons and pions overlap in # of layers traveled and with diff distributions

### TODO
- Transform objective values?
- Better pareto front
- Outer radius:
    - objective constraint?
    - don't calculate as black box?
    - don't use as objective?


Types of graphs:
- pareto front:
    - 3d graphs
        - https://www.desmos.com/3d/hxmrut81bs
        - https://www.desmos.com/3d/fovj0otenz
    - 2d graphs with third objective color scale
- param vs output:
    - contour plots as slices


Note that PID algorithm assumes muons and pions in equal numbers

In [1]:
# For each layer, in mm
airgap = 0.3
scint = 20
steel = 55.5

one_layer = airgap * 2 + scint + steel
n_layers = 14
all_layers = n_layers * one_layer

inner_radius = 1770
outer_radius = inner_radius + all_layers
outer_radius

2835.3999999999996

In [1]:
import plotly.io as pio
pio.renderers.default = 'iframe'

import sys
sys.path.append('/hpc/group/vossenlab/rck32/eic/dRICH-MOBO/MOBO-tools')
eic_pref = "/hpc/group/vossenlab/rck32/eic/"
from wrapper_slurm_basic import *

from ax.plot.scatter import interact_fitted, lattice_multiple_metrics
from ax.plot.pareto_utils import compute_posterior_pareto_frontier, get_observed_pareto_frontiers
from ax.plot.pareto_frontier import scatter_plot_with_hypervolume_trace_plotly, plot_pareto_frontier, interact_pareto_frontier
from ax.plot.slice import plot_slice
from ax.plot.slice import interact_slice
from ax.plot.contour import interact_contour
from ax.plot.contour import plot_contour
from ax.modelbridge.cross_validation import compute_diagnostics, cross_validate
from ax.plot.diagnostic import interact_cross_validation
from ax.plot.parallel_coordinates import plot_parallel_coordinates
from ax.utils.notebook.plotting import render

from ax.storage.json_store.load import load_experiment
bundle = RegistryBundle(metric_clss={SlurmJobMetric: None}, runner_clss={SlurmJobRunner: None})


from ax.plot.render import plot_config_to_html
from ax.utils.report.render import render_report_elements
import plotly.graph_objs as go
import os
import shutil

In [24]:
class PlotGenerator:
    def __init__(self,experiment_dir, experiment_info):
        self.experiment_dir = experiment_dir
        self.experiment_info = experiment_info
        self.zoom = "tight"
        if(self.zoom == "loose"):
            self.plot_dir = "/hpc/group/vossenlab/rck32/eic/dRICH-MOBO/MOBO-tools/plots/PaperPlots_wide_axes/" #loose
        else:
            self.plot_dir = "/hpc/group/vossenlab/rck32/eic/dRICH-MOBO/MOBO-tools/plots/PaperPlots_zoomed/" #tight
        
        self.info_to_day_dict = {
            "may_14__objectives__high_rmse_1GeV_mupi_may_14__parameters__division_layer_number_preshower_steel_value" : "_may_14",
            "may_13__objectives__low_rmse_5GeV_mupi_may_13__parameters__division_layer_number_preshower_steel_value" : "_may_13",
            "may_11__objectives__low_rmse_1GeV_mupi_may_11__parameters__division_layer_number_preshower_steel_value": "_may_11",
            "may_10__objectives__high_rmse_5GeV_mupi_may_10__parameters__division_layer_number_preshower_steel_value": "_may_10",
            "may_8__objectives__low_rmse_high_rmse_may_8__parameters__division_layer_number_preshower_steel_value": "_may_8",
            "april_26__objectives__high_rmse_low_rmse_5GeV_mupi_1GeV_mupi_april_26__parameters__preshower_ratio_preshower_steel_ratio_postshower_steel_ratio" : "_april_26",
            "april_18__objectives__high_rmse_low_rmse_1GeV_mupi_april_18__parameters__num_layers_steel_ratio_thick_diff_ratio": "_april_18",
            "april_4__objectives__high_rmse_low_rmse_5GeV_mupi_1GeV_mupi_april_4__parameters__num_layers_steel_ratio" : "_april_4",
            "june_26__objectives__high_rmse_low_rmse_june_26__parameters__steel_slope_scint_slope_steel_ratio" : "_june_26",
            "june_27__objectives__high_rmse_high_mupi_june_27__parameters__steel_slope_scint_slope_steel_ratio" : "_june_27",
            "june_28__objectives__low_rmse_low_mupi_june_28__parameters__steel_slope_scint_slope_steel_ratio" : "_june_28",
            "june_29__objectives__low_rmse_high_mupi_june_29__parameters__steel_slope_scint_slope_steel_ratio" : "_june_29",
            "june_30__objectives__high_rmse_low_mupi_june_30__parameters__steel_slope_scint_slope_steel_ratio" : "_june_30"
        }
        self.date_suffix = self.info_to_day_dict[self.experiment_info]
        
        '''Members for naming'''
        
        '''Members for axes limits'''
        self.axes_lims_obj_dict_loose = {
            #relavent to all experiments
            'low_RMSE_pareto' : [0.4, 0.81],
            'low_RMSE_slice' : [0.44,0.86],
            'high_RMSE_pareto' : [0.6,0.9],
            'high_RMSE_slice' : [0.64,1.05],
            'sepMuPi_1GeV_pareto' : [0.625,0.975],
            'sepMuPi_1GeV_slice' : [0.6,0.975],
            'sepMuPi_5GeV_pareto' : [0.69,0.935],
            'sepMuPi_5GeV_slice' : [0.475,0.95],
        }
        self.axes_lims_obj_dict_tight = {
            #April 18
            'low_RMSE_pareto_april_18' : [0.42,0.565],
            'low_RMSE_slice_april_18' : [0.45,0.77],
            'high_RMSE_pareto_april_18' : [0.6,0.775],
            'high_RMSE_slice_april_18' : [0.65,1.05],
            'sepMuPi_1GeV_pareto_april_18' : [0.86,0.975],
            'sepMuPi_1GeV_slice_april_18' : [0.4,1],
            
            #April 26
            'low_RMSE_pareto_april_26' : [0.45,0.81],
            'low_RMSE_slice_april_26' : [0.45,1],
            'high_RMSE_pareto_april_26' : [0.64,0.9],
            'high_RMSE_slice_april_26' : [0.675,1],
            'sepMuPi_1GeV_pareto_april_26' : [0.625,0.97],
            'sepMuPi_1GeV_slice_april_26' : [0.6,1],
            'sepMuPi_5GeV_pareto_april_26' : [0.69,0.93],
            'sepMuPi_5GeV_slice_april_26' : [0.55,0.95],
            
            #April 4
            'low_RMSE_pareto_april_4' : [0.425,0.59],
            'low_RMSE_slice_april_4' : [0.45,0.825],
            'high_RMSE_pareto_april_4' : [0.61,0.745],
            'high_RMSE_slice_april_4' : [0.65,1.025],
            'sepMuPi_1GeV_pareto_april_4' : [0.86,0.96],
            'sepMuPi_1GeV_slice_april_4' : [0.595,0.96],
            'sepMuPi_5GeV_pareto_april_4' : [0.695,0.935],
            'sepMuPi_5GeV_slice_april_4' : [0.475,0.94],
            
            #May 10
            'high_RMSE_pareto_may_10' : [0.656,0.892],
            'high_RMSE_slice_may_10' : [0.65,0.775],
            'sepMuPi_5GeV_pareto_may_10' : [0.795,0.88],
            'sepMuPi_5GeV_slice_may_10' : [0.77,0.85],
            
            #May 11
            'low_RMSE_pareto_may_11' : [0.445,0.55],
            'low_RMSE_slice_may_11' : [0.45,0.6],
            'sepMuPi_1GeV_pareto_may_11' : [0.905,0.95],
            'sepMuPi_1GeV_slice_may_11' : [0.89,0.95],
            
            #May 13
            'low_RMSE_pareto_may_13' : [0.42,0.615],
            'low_RMSE_slice_may_13' : [0.445,0.61],
            'sepMuPi_5GeV_pareto_may_13' : [0.815,0.87],
            'sepMuPi_5GeV_slice_may_13' : [0.76,0.84],
            
            #May 14
            'high_RMSE_pareto_may_14' : [0.64,0.74],
            'high_RMSE_slice_may_14' : [0.67,0.79],
            'sepMuPi_1GeV_pareto_may_14' : [0.88,0.95],
            'sepMuPi_1GeV_slice_may_14' : [0.86,0.94],
            
            #May 8
            'low_RMSE_pareto_may_8' : [0.41,0.56],
            'low_RMSE_slice_may_8' : [0.44,0.86],
            'high_RMSE_pareto_may_8' : [0.64,0.86],
            'high_RMSE_slice_may_8' : [0.64,0.825],
            
            #June 26
            'low_RMSE_pareto_june_26' : [0.42,0.55],
            'low_RMSE_slice_june_26' : [0.45,0.71],
            'high_RMSE_pareto_june_26' : [0.64,0.71],
            'high_RMSE_slice_june_26' : [0.65,0.98],
            
            #June 27
            'high_RMSE_pareto_june_27' : [0.62,0.71],
            'high_RMSE_slice_june_27' : [0.66,0.97],
            'sepMuPi_5GeV_pareto_june_27' : [0.85,1.0],
            'sepMuPi_5GeV_slice_june_27' : [0.45,0.91],
            
            #June 28
            'low_RMSE_pareto_june_28' : [0.44,0.55],
            'low_RMSE_slice_june_28' : [0.45,0.75],
            'sepMuPi_1GeV_pareto_june_28' : [0.87,1.0],
            'sepMuPi_1GeV_slice_june_28' : [0.64,0.95],
            
            #June 29
            'low_RMSE_pareto_june_29' : [0.45,0.52],
            'low_RMSE_slice_june_29' : [0.45,0.65],
            'sepMuPi_5GeV_pareto_june_29' : [0.8,1.0],
            'sepMuPi_5GeV_slice_june_29' : [0.44,0.9],
            
            #June 30
            'high_RMSE_pareto_june_30' : [0.64,0.75],
            'high_RMSE_slice_june_30' : [0.65,1.0],
            'sepMuPi_1GeV_pareto_june_30' : [0.9,1.0],
            'sepMuPi_1GeV_slice_june_30' : [0.62,0.97]
        }
        
        self.axes_lims_param_dict = {
            #relavent to may 8-14 experiments
            'preshower_steel_value' : [0,115],
            'division_layer_number' : [0,8],
            #relevant to april 26 preshower experiment
            'preshower_ratio' : [0.04,0.75],
            'preshower_steel_ratio' : [0.05,0.95],
            'postshower_steel_ratio' : [0.05,0.95],
            #relevant to april 18 linear ratio
            'steel_ratio' : [0,1],
            'num_layers' : [4,19],
            'thick_diff_ratio' : [-1,1],
            #relevant to june experiments
            'steel_slope' : [-1,1],
            'scint_slope' : [-1,1]
        }
        self.yaxis_label_dict = {
            'high_RMSE' : 'High energy RMSE (GeV)',
            'low_RMSE' : 'Low energy RMSE (GeV)',
            'sepMuPi_1GeV' : '1GeV Muon/Pion separation ROC AUC',
            'sepMuPi_5GeV' : '5GeV Muon/Pion separation ROC AUC'
        }
        self.xaxis_label_dict = {
            'preshower_steel_value'  : 'Pre shower steel thickness (mm)',
            'division_layer_number'  : 'Pre shower dividing layer number',
            #preshower_ratio: ratio  of total thickness of "preshower" section to "postshower" section of KLM
            'preshower_ratio'        : 'Pre- vs post- shower ratio',
            #preshower_steel_ratio:  ratio of total preshower thickness that is steel (vs scint)
            'preshower_steel_ratio'  : "pre-shower steel ratio",
            'postshower_steel_ratio' : "post-shower steel ratio",
            'steel_ratio' : 'Steel to scintillator ratio',
            'num_layers' : 'Number of layers',
            'thick_diff_ratio' : 'Layer thickness factor',
            'scint_slope' : 'Scintillator thickness slope',
            'steel_slope' : 'Steel thickness slope'
        }
        self.pareto_front_axis_font_size = 18
        self.process()
    def create_dir(self):
        paths = [f"{self.plot_dir}SlicePlots/{self.experiment_info}",f"{self.plot_dir}ParetoFront/{self.experiment_info}"]
        for path in paths:
            if not os.path.isdir(path):
                os.mkdir(path)
    def get_axes_lims_param(self,param):
        return self.axes_lims_param_dict[param]
    def get_axes_lims_obj(self,obj,plot_type):
        if(self.zoom == "loose"):
            return self.axes_lims_obj_dict_loose[obj + "_" + plot_type]
        else:
            return self.axes_lims_obj_dict_tight[obj + "_" + plot_type + self.date_suffix]
    def process(self):
        self.experiment = load_experiment(filepath=eic_pref + 'dRICH-MOBO/MOBO-tools/' + self.experiment_dir + '/test_scheduler_experiment.json', decoder_registry=bundle.decoder_registry)
        
        if(self.experiment_dir == "June_26_experiment"):
            new_objective_thresholds = [
                ObjectiveThreshold(metric=metrics[0], bound=0.6, relative=False), #high energy RMSE: 0.75
                ObjectiveThreshold(metric=metrics[1], bound=0.75, relative=False)#, #low energy RMSE: 0.6
                ]
            self.experiment.optimization_config.objective_thresholds = new_objective_thresholds
        
        with open(eic_pref + 'dRICH-MOBO/MOBO-tools/' + self.experiment_dir + '/gs_model.pkl', 'rb') as file:
            self.model = pickle.load(file)
        self.data = self.experiment.fetch_data()
        
        self.objectives = self.experiment.optimization_config.objective.objectives
        self.objectives_names = [x.metric.name for x in self.objectives]
        
        self.parameters = self.experiment.parameters
        self.parameters_names = list(self.parameters.keys())
        
        self.absolute_metrics = list(self.experiment.metrics.keys())
        self.num_points = 60
        
        self.yaxis_labels = [self.yaxis_label_dict[name] for name in self.objectives_names]
        self.xaxis_labels = [self.xaxis_label_dict[name] for name in self.parameters_names]
        
        '''Y axis is objective, x axis parameter'''
        self.xaxis_lims = [self.get_axes_lims_param(param_name) for param_name in self.parameters_names]
        self.yaxis_lims = [self.get_axes_lims_obj(objective_name,"slice") for objective_name in self.objectives_names]
        
        self.create_dir()
        
    def plot_slices(self):
        for iter_params, parameter_name in enumerate(self.parameters_names):
            for iter_objectives, objective_name in enumerate(self.objectives_names):

                slice_plot = plot_slice(self.model, parameter_name,objective_name)

                slice_plot.data['layout']['xaxis']['title']['text'] = self.xaxis_labels[iter_params]
                slice_plot.data['layout']['yaxis']['title']['text'] = self.yaxis_labels[iter_objectives]
                slice_plot.data['layout']['yaxis']['title']['font'] = {'size': 18}
                slice_plot.data['layout']['xaxis']['title']['font'] = {'size': 18}
                slice_plot.data['layout']['xaxis']['range'] = self.xaxis_lims[iter_params]
                slice_plot.data['layout']['yaxis']['range'] = self.yaxis_lims[iter_objectives]
                slice_plot.data['layout']['title'] = {}
                fig = go.Figure(data=slice_plot.data['data'],layout=slice_plot.data['layout'])
                pio.write_image(fig,f"{self.plot_dir}SlicePlots/{self.experiment_info}/objective_{self.objectives_names[iter_objectives]}_parameter_{parameter_name}.pdf", format="pdf")
    def plot_pareto_fronts(self):
        observed_frontiers = get_observed_pareto_frontiers(experiment=self.experiment,rel=False)
        
        for i, observed_frontier in enumerate(observed_frontiers):

            primary_metric = observed_frontier.primary_metric #y axis
            secondary_metric = observed_frontier.secondary_metric #x axis

            pf = plot_pareto_frontier(observed_frontier)
            pf.data['layout']['shapes'] = []
            fig = go.Figure(data=pf.data['data'], layout=pf.data['layout'])
            fig.update_layout(
                title_text=f"",
                xaxis_title=dict(
                    text=self.yaxis_label_dict[secondary_metric],
                    font=dict(size=self.pareto_front_axis_font_size)  # Set x-axis title font size
                ),
                yaxis_title=dict(
                    text=self.yaxis_label_dict[primary_metric],
                    font=dict(size=self.pareto_front_axis_font_size)  # Set y-axis title font size
                ),
                width=600,  # or try 600
                height=400,
                margin=dict(l=50, r=50, t=50, b=50),
                xaxis_range=self.get_axes_lims_obj(secondary_metric, "pareto"),
                yaxis_range=self.get_axes_lims_obj(primary_metric, "pareto")
            )
            pio.write_image(fig, f"{self.plot_dir}/ParetoFront/{self.experiment_info}/primary_{primary_metric}__vs__secondary_{secondary_metric}.pdf", format="pdf")

## June 30th

In [25]:
# USING PlotGenerator
pg_june30 = PlotGenerator("experiment_high_rmse_low_mupi_june_30","june_30__objectives__high_rmse_low_mupi_june_30__parameters__steel_slope_scint_slope_steel_ratio")
pg_june30.plot_slices()
pg_june30.plot_pareto_fronts()

[WARNING 07-02 13:05:02] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


In [33]:

observed_frontiers = get_observed_pareto_frontiers(experiment=pg_june30.experiment,rel=False)
pareto_front_report = interact_pareto_frontier(observed_frontiers)
render(pareto_front_report)

## June 29th

In [26]:
# USING PlotGenerator
pg_june29 = PlotGenerator("experiment_low_rmse_high_mupi_june_29","june_29__objectives__low_rmse_high_mupi_june_29__parameters__steel_slope_scint_slope_steel_ratio")
pg_june29.plot_slices()
pg_june29.plot_pareto_fronts()

[WARNING 07-02 13:05:07] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


In [32]:

observed_frontiers = get_observed_pareto_frontiers(experiment=pg_june29.experiment,rel=False)
pareto_front_report = interact_pareto_frontier(observed_frontiers)
render(pareto_front_report)

## June 28th

In [27]:
# USING PlotGenerator
pg_june28 = PlotGenerator("experiment_low_rmse_low_mupi_june_28","june_28__objectives__low_rmse_low_mupi_june_28__parameters__steel_slope_scint_slope_steel_ratio")
pg_june28.plot_slices()
pg_june28.plot_pareto_fronts()

[WARNING 07-02 13:05:10] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


In [31]:

observed_frontiers = get_observed_pareto_frontiers(experiment=pg_june28.experiment,rel=False)
pareto_front_report = interact_pareto_frontier(observed_frontiers)
render(pareto_front_report)

## June 27th

In [28]:
# USING PlotGenerator
pg_june27 = PlotGenerator("experiment_high_rmse_high_mupi_june_27","june_27__objectives__high_rmse_high_mupi_june_27__parameters__steel_slope_scint_slope_steel_ratio")
pg_june27.plot_slices()
pg_june27.plot_pareto_fronts()

[WARNING 07-02 13:05:12] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


In [30]:

observed_frontiers = get_observed_pareto_frontiers(experiment=pg_june27.experiment,rel=False)
pareto_front_report = interact_pareto_frontier(observed_frontiers)
render(pareto_front_report)

## June 26th

In [29]:
# USING PlotGenerator
pg_june26 = PlotGenerator("experiment_high_rmse_low_rmse_june_26","june_26__objectives__high_rmse_low_rmse_june_26__parameters__steel_slope_scint_slope_steel_ratio")
pg_june26.plot_slices()
pg_june26.plot_pareto_fronts()

[WARNING 07-02 13:05:14] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


In [14]:

observed_frontiers = get_observed_pareto_frontiers(experiment=pg_june26.experiment,rel=False)
pareto_front_report = interact_pareto_frontier(observed_frontiers)
render(pareto_front_report)

## May 14: Pre shower version 2 (high energy neutron and 1GeV mu/pi)

In [71]:
# USING PlotGenerator
pg_may14 = PlotGenerator("experiment_high_rmse_low_mupi_may_14","may_14__objectives__high_rmse_1GeV_mupi_may_14__parameters__division_layer_number_preshower_steel_value")
pg_may14.plot_slices()
pg_may14.plot_pareto_fronts()

[WARNING 06-22 20:01:50] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


In [85]:

observed_frontiers = get_observed_pareto_frontiers(experiment=pg_may14.experiment,rel=False)
pareto_front_report = interact_pareto_frontier(observed_frontiers)
render(pareto_front_report)

## May 13: Pre shower version 2 (low energy neutron and 5GeV mu/pi)

In [72]:
pg_may13 = PlotGenerator("experiment_low_rmse_high_mupi_may_13","may_13__objectives__low_rmse_5GeV_mupi_may_13__parameters__division_layer_number_preshower_steel_value")
pg_may13.plot_slices()
pg_may13.plot_pareto_fronts()

[WARNING 06-22 20:01:51] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


In [84]:

observed_frontiers = get_observed_pareto_frontiers(experiment=pg_may13.experiment,rel=False)
pareto_front_report = interact_pareto_frontier(observed_frontiers)
render(pareto_front_report)

## May 11: Pre shower version 2 (low energy neutron and mu/pi)

In [73]:
pg_may11 = PlotGenerator("experiment_preshower_steel_low_n_low_mupi_may_11","may_11__objectives__low_rmse_1GeV_mupi_may_11__parameters__division_layer_number_preshower_steel_value")
pg_may11.plot_slices()
pg_may11.plot_pareto_fronts()

[WARNING 06-22 20:01:53] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


In [83]:

observed_frontiers = get_observed_pareto_frontiers(experiment=pg_may11.experiment,rel=False)
pareto_front_report = interact_pareto_frontier(observed_frontiers)
render(pareto_front_report)

## May 10: pre shower version 2 (both high energy)

In [74]:
pg_may10 = PlotGenerator("Experiment_high_mu_energy_may_10","may_10__objectives__high_rmse_5GeV_mupi_may_10__parameters__division_layer_number_preshower_steel_value")
pg_may10.plot_slices()
pg_may10.plot_pareto_fronts()

[WARNING 06-22 20:01:54] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


In [81]:

observed_frontiers = get_observed_pareto_frontiers(experiment=pg_may10.experiment,rel=False)
pareto_front_report = interact_pareto_frontier(observed_frontiers)
render(pareto_front_report)

## May 8-9th: Pre shower version 2 (2 neutron obj, 2 params)

In [75]:
pg_may8 = PlotGenerator("experiment_may_8_preshower_v2_both_neutron","may_8__objectives__low_rmse_high_rmse_may_8__parameters__division_layer_number_preshower_steel_value")
pg_may8.plot_slices()
pg_may8.plot_pareto_fronts()

[WARNING 06-22 20:01:56] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


In [82]:

observed_frontiers = get_observed_pareto_frontiers(experiment=pg_may8.experiment,rel=False)
pareto_front_report = interact_pareto_frontier(observed_frontiers)
render(pareto_front_report)

# Pre-shower

In [76]:
pg_april26 = PlotGenerator("experiment_april_26_preshower_v1","april_26__objectives__high_rmse_low_rmse_5GeV_mupi_1GeV_mupi_april_26__parameters__preshower_ratio_preshower_steel_ratio_postshower_steel_ratio")
pg_april26.plot_slices()
pg_april26.plot_pareto_fronts()

[WARNING 06-22 20:01:57] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


# Linear ratio (non-uniform)

In [77]:
pg_april_18 = PlotGenerator("experiment_april_18_linear_ratio","april_18__objectives__high_rmse_low_rmse_1GeV_mupi_april_18__parameters__num_layers_steel_ratio_thick_diff_ratio")
pg_april_18.plot_slices()
pg_april_18.plot_pareto_fronts()

[WARNING 06-22 20:02:02] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.


## April 4 Basic experiment (steel ratio and number of layers only)

In [78]:
pg_april_4 = PlotGenerator("experiment_april_4_basic","april_4__objectives__high_rmse_low_rmse_5GeV_mupi_1GeV_mupi_april_4__parameters__num_layers_steel_ratio")
pg_april_4.plot_slices()
pg_april_4.plot_pareto_fronts()

[WARNING 06-22 20:02:06] ax.core.experiment: Experiment's status_quo is updated. Generally the status_quo should not be changed after being set.
